# Keras and Optuna hyperparameter optimization on MNIST dataset

In [1]:
# importing libraries

from tensorflow.keras.backend import clear_session
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

import optuna

In [2]:
# importing data

(x_train, y_train), (x_valid, y_valid) = mnist.load_data()

In [3]:
# setting global variables


N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 10

In [4]:
# reshaping imgs

img_x, img_y = x_train.shape[1], x_train.shape[2]
x_train = x_train.reshape(-1, img_x, img_y, 1)[:N_TRAIN_EXAMPLES].astype("float32") / 255
x_valid = x_valid.reshape(-1, img_x, img_y, 1)[:N_VALID_EXAMPLES].astype("float32") / 255
y_train = y_train[:N_TRAIN_EXAMPLES]
y_valid = y_valid[:N_VALID_EXAMPLES]
input_shape = (img_x, img_y, 1)

In [5]:
# defining "objective" function of Oputna

def objective(trial):
    
    # defining model
    model = Sequential()
    model.add(
            Conv2D(
            filters=trial.suggest_categorical("filters", [32, 64]),
            kernel_size=trial.suggest_categorical("kernel_size", [3, 5]),
            strides=trial.suggest_categorical("strides", [1, 2]),
            activation=trial.suggest_categorical("activation", ["relu", "linear", 'swish']),
            input_shape=input_shape,
            )
        )
    model.add(Flatten())
    model.add(Dense(CLASSES, activation="softmax"))

    # defining suggestion for learning rate and optimizer
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])

    if optimizer_name=="Adam":
        optimizer = Adam(learning_rate=lr)
    elif optimizer_name=="RMSprop":
        optimizer = RMSprop(learning_rate=lr)
    elif optimizer_name=="SGD":
        optimizer = SGD(learning_rate=lr)
    
    # compiling model
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]), metrics=["accuracy"]
    )

    #fitting model
    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )

    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]

In [7]:
# create Optuna study

study = optuna.create_study(direction="maximize", sampler=TPESampler(multivariate=True), pruner=HyperbandPruner)
study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

/root/venv/lib/python3.7/site-packages/optuna/samplers/_tpe/sampler.py:191: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  ExperimentalWarning,
[I 2021-03-01 13:37:40,254] A new study created in memory with name: no-name-261da559-fa0c-4943-b613-aabdfda75268
[I 2021-03-01 13:37:46,808] Trial 0 finished with value: 0.8870000243186951 and parameters: {'filters': 64, 'kernel_size': 3, 'strides': 2, 'activation': 'swish', 'lr': 0.022340008699868515, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.8870000243186951.
[I 2021-03-01 13:37:57,146] Trial 1 finished with value: 0.9139999747276306 and parameters: {'filters': 32, 'kernel_size': 3, 'strides': 1, 'activation': 'relu', 'lr': 0.003004629430809697, 'optimizer': 'Adam'}. Best is trial 1 with value: 0.9139999747276306.
[I 2021-03-01 13:38:26,378] Trial 2 finished with value: 0.8690000176429749 and parameters: {'filters': 64, 'kernel_size': 3, 'strides': 1, 'activation':

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9f2954cd-f149-4d4f-a4b0-a1e3eb6d921f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>